In [1]:
# Initiate system
# ------------------------------------------------------------
import pandas as pd 

# Load data as pickle
df_Series = pd.read_pickle(r"C:\Users\beckm\Documents\GitHub\rvbrtg\Modaliteter\Konventionell\Fysiker-Projekt\Data_24-12-05\series_data.pkl")
df_Study = pd.read_pickle(r"C:\Users\beckm\Documents\GitHub\rvbrtg\Modaliteter\Konventionell\Fysiker-Projekt\Data_24-12-05\study_data.pkl")

# Merge data based on studyInstanceUID
merged = pd.merge(df_Series.reset_index(), df_Study.reset_index(), on='studyInstanceUID', how="left")

# Filter merged data, remove all non-relevant protocols,
    # I've hard-coded this filtration based on all unique protocols.. It's not pretty but works and provides a list of all protocols to calc separately (later)
ft_merged = merged[~merged["acquisitionProtocol"].isin(['Position Skellefteå','-----','---Frigör systemet---','.OL Benlängd 20-40kg belastad','.OL Benlängd belastad','.OL Benvinkel DX 20-40kg belastad',
                                                '.OL Benvinkel DX 20-50kg belastad','.OL Benvinkel DX belastad','.OL Benvinkel DX stående','.OL Benvinkel SIN 20-50kg belastad','.OL Benvinkel SIN belastad',
                                                '.OL Benvinkel SIN stående','.OL Helrygg frontal AP 0-20kg','.OL Helrygg frontal AP 20-50kg','.OL Helrygg frontal AP vuxen','.OL Helrygg frontal PA 0-20kg',
                                                '.OL Helrygg frontal PA 20-50kg','.OL Helrygg frontal PA vuxen','.OL Helrygg sida 0-20kg','.OL Helrygg sida 20-50kg','.OL Helrygg sida stag 20-50kg',
                                                '.OL Helrygg sida vuxen','.OL Underben DX frontal belastad','.OL Underben SIN frontal belastad','.OL Underben SIN sida belastad','.OT Helrygg frontal',
                                                '.OT Helrygg frontal 20-50kg','.OT Helrygg frontal vuxen','.OT Lårben DX frontal','.OT Lårben SIN frontal','.OT Tunntarmspassage','.OT Underben DX frontal',
                                                '.OT Underben DX sida','.OT Underben SIN frontal','.OT Underben SIN sida','CP_Shunt','CP_Vuxen esofagus','CP_antiiso','OL Benvinkel DX stående',
                                                'OL Benvinkel SIN stående','OL Helrygg frontal PA vuxen','OL Helrygg sida vuxen','OT Buköversikt','OT Helrygg frontal vuxen','OT Helrygg sida vuxen',
                                                'OT Lårben DX frontal','OT Lårben SIN frontal','OT Tunntarm','OT Tunntarmpassage','OT Underben DX frontal','OT Underben DX sida','OT Underben SIN frontal',
                                                'OT Underben SIN sida','OW Benlängd belastad','OW Benvinkel DX','OW Benvinkel DX 20-40 kg','OW Benvinkel SIN',"OW Helrygg frontal 20 -30 kg", 
                                                "OW Helrygg frontal 30-40 kg", "OW Helrygg frontal 40-50 kg", "OW Helrygg frontal 50-60 kg", "OW Helrygg frontal 60-70 kg", "OW Helrygg frontal vuxen", 
                                                "OW Helrygg sida 20-30 kg", "OW Helrygg sida 30-40 kg", "OW Helrygg sida 40-50 kg", "OW Helrygg sida 50-60 kg", "OW Helrygg sida 60-70 kg", 
                                                "OW Helrygg sida vuxen", "OW Skalle-halsrygg-bröstrygg frontal", "OW Skalle-halsrygg-bröstrygg sida",'Sondläge','____________________'])]

In [13]:
# Match data
# ------------------------------------------------------------

# Match and identify nr of duplicate acq.prot for each studyUID, 
duplicate_acqprot = ft_merged.pivot_table(
    index='studyInstanceUID',
    values='acquisitionProtocol',
    aggfunc=[
        lambda x: x.duplicated().sum(),
        'count'
    ]
)

# Rename columns
duplicate_acqprot.columns = ['duplicates', 'total_count']

# Extract data from DataFrame to 
duplicates_dup = duplicate_acqprot['duplicates']
duplicates_tot = duplicate_acqprot['total_count']
duplicates_rat = round(100*(duplicate_acqprot['duplicates'] / duplicate_acqprot['total_count']))
duplicates_rat.name = "Ratio"

# Then perform the merge
joined_ft_merged = pd.merge(ft_merged, duplicates_dup, on='studyInstanceUID', how="left")
joined_ft_merged = pd.merge(joined_ft_merged, duplicates_tot, on='studyInstanceUID', how="left")
joined_ft_merged = pd.merge(joined_ft_merged.reset_index(), duplicates_rat.reset_index(), on='studyInstanceUID', how="left")


# Test to identify nr of extra acq.prot due to re-takes
duplicate_UID = ft_merged.pivot_table(
    index='studyDescription',
    values='studyInstanceUID',
    aggfunc=[
        lambda x: x.duplicated().sum(),
        'count'
    ]
)

# Rename columns
duplicate_UID.columns = ['duplicates', 'total_count']

# Print for de-bug
#print(duplicates)

In [3]:
# Estimate ratios of interest from dataset
# ------------------------------------------------------------

# Ratio, nr of examinations that include one or more re-takes,
P_exam = round(100*(duplicate_acqprot['duplicates'] >0 ).sum()/len(duplicate_acqprot))
print(f"About {P_exam} % of all examinations have one or more retake images")

# Ratio, tot nr of extra exposures due to re-takes,
P_expo = round(100*(duplicate_acqprot['duplicates'].sum()/duplicate_acqprot['total_count'].sum()))
print(f"About {P_expo} % of all exposures are retakes")

# Estimate percentage for a acq.prot to require one or more re-takes to complete
duplicate_UID['duplicate_percentage'] = round(100*(duplicate_UID['duplicates']/duplicate_UID['total_count']))
print(duplicate_UID['duplicate_percentage'])

About 33 % of all examinations have one or more retake images
About 16 % of all exposures are retakes
studyDescription
.FOT DX            67.0
.HÖFTLED DX        94.0
.HÖFTLED SIN        0.0
.KNÄLED DX         60.0
ANSIKSSKELETT       0.0
                   ... 
Öra cochlea DX      0.0
Öra cochlea SIN    40.0
Överarm            75.0
Överarm DX         63.0
Överarm SIN        62.0
Name: duplicate_percentage, Length: 99, dtype: float64


In [ ]:
# Sample DataFrame
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'Score': [85, 90, 95]
})

# Extracting a column
series1 = df['Age']  # Series with just the 'Age' column

# Flattening the entire DataFrame
series2 = df.stack()

# Extracting a row
series3 = df.loc[1]  # Row with index 1 (Bob)

# Combining all values
series4 = pd.Series(df.values.flatten())

print(df)
print(series1)
#print(series2)
#print(series3)
#print(series4)


      Name  Age  Score
0    Alice   25     85
1      Bob   30     90
2  Charlie   35     95
0    25
1    30
2    35
Name: Age, dtype: int64
